In [15]:
import pandas as pd
import json as js
from functools import reduce
import datetime as dt

In [16]:
# open json file
with open("Resources/kossal_copy_resources/all_electricity_supply.json") as supply_json:
    supply = js.load(supply_json)

In [17]:
# find number of columns from the data fram
num_of_col = len(supply)
num_of_col

12

In [18]:
s='Net generation : coal : United States : all sectors : monthly'
start='Net generation : '
end=' : United States : all sectors : monthly'
text=s[s.find(start)+len(start):s.rfind(end)]

In [19]:
# find columns names string and get the power sources
# supply[0]['name'] result in string 'Net generation : coal : United States : all sectors : monthly'
col_name = []
for i in range(len(supply)):
    s=supply[i]['name']    
    col_name.append(str(s[s.find(start)+len(start):s.rfind(end)]))
col_name

['coal',
 'natural gas',
 'nuclear',
 'conventional hydroelectric',
 'wind',
 'petroleum liquids',
 'all utility-scale solar',
 'geothermal',
 'wood and wood-derived fuels',
 'other gases',
 'other biomass',
 'other']

In [20]:
# create list of dataFrame Name
df_name=['df_'+str(i) for i in range(len(supply))]
df_name

['df_0',
 'df_1',
 'df_2',
 'df_3',
 'df_4',
 'df_5',
 'df_6',
 'df_7',
 'df_8',
 'df_9',
 'df_10',
 'df_11']

In [21]:
# Use for loop to create data frames
for index, col in enumerate(col_name):
       exec(f"{df_name[index]} = pd.DataFrame({supply[index]['data']}, columns=['date_id', '{col}'])")    

In [22]:
# create a list of DataFrames for join loop
df_list = [eval(temp) for temp in df_name]

In [23]:
# join DataFrames
df_energy_source = reduce(lambda x, y: pd.merge(x,y, on = 'date_id'), df_list)
df_energy_source.head()

,date_id,coal,natural gas,nuclear,conventional hydroelectric,wind,petroleum liquids,all utility-scale solar,geothermal,wood and wood-derived fuels,other gases,other biomass,other
0,202009,68395.67702,141406.66981,65727.317,19182.11175,23176.03240,794.67219,7757.40868,1390.39750,2938.04250,947.17872,1512.22945,1006.84768
1,202008,91164.99383,174129.84618,68982.187,24227.83917,22570.71305,925.96988,9369.91528,1426.23924,3302.49204,1002.10601,1589.15438,1096.05539
2,202007,89845.33355,185433.35809,69385.440,27753.19342,22579.21085,963.15441,10379.27930,1430.94455,3111.53745,853.95303,1564.17641,1088.60389
3,202006,65274.37394,143179.90065,67205.083,29137.98926,29871.72279,864.67926,9529.17251,1369.44148,2993.49186,756.49600,1458.91919,1016.00390
4,202005,46529.48478,115854.51716,64337.970,30559.05329,28163.26916,707.14715,9734.37493,1448.82619,3099.05893,807.66622,1616.93726,1088.42162


In [24]:
# rename column to for SQL Tables
df_energy_source=df_energy_source.rename(columns={'natural gas':'natural_gas', 
                                 'conventional hydroelectric':'hydroelectric',
                                 'petroleum liquids':'petroleum_liquids',
                                 'all utility-scale solar': 'solar',
                                 'wood and wood-derived fuels':'wood',
                                 'other gases':'other_gases',
                                 'other biomass': 'other_biomass'
                                })
df_energy_source.head()

,date_id,coal,natural_gas,nuclear,hydroelectric,wind,petroleum_liquids,solar,geothermal,wood,other_gases,other_biomass,other
0,202009,68395.67702,141406.66981,65727.317,19182.11175,23176.03240,794.67219,7757.40868,1390.39750,2938.04250,947.17872,1512.22945,1006.84768
1,202008,91164.99383,174129.84618,68982.187,24227.83917,22570.71305,925.96988,9369.91528,1426.23924,3302.49204,1002.10601,1589.15438,1096.05539
2,202007,89845.33355,185433.35809,69385.440,27753.19342,22579.21085,963.15441,10379.27930,1430.94455,3111.53745,853.95303,1564.17641,1088.60389
3,202006,65274.37394,143179.90065,67205.083,29137.98926,29871.72279,864.67926,9529.17251,1369.44148,2993.49186,756.49600,1458.91919,1016.00390
4,202005,46529.48478,115854.51716,64337.970,30559.05329,28163.26916,707.14715,9734.37493,1448.82619,3099.05893,807.66622,1616.93726,1088.42162


In [25]:
# columns of interest
col_int = ['date_id', 'natural_gas', 'nuclear', 'hydroelectric', 'wind', 'solar', 'wood', 'other_biomass', 'other']

In [27]:
# select column of interest
df_ener_sour = df_energy_source[col_int]
df_ener_sour = df_ener_sour.sort_values(by=['date_id'], ascending = True).reset_index(drop=True)
df_ener_sour.head()

,date_id,natural_gas,nuclear,hydroelectric,wind,solar,wood,other_biomass,other
0,200101,42388.663,68707.077,18852.048,389.250,6.500,3191.212,1208.64715,991.66885
1,200102,37966.927,61272.407,17472.889,431.242,12.568,2697.151,1096.33055,871.94945
2,200103,44364.414,62140.712,20477.189,532.120,31.498,2852.540,1189.04028,931.15472
3,200104,45842.746,56003.026,18012.994,684.695,38.759,2821.007,1187.23919,957.45581
4,200105,50934.205,61512.445,19175.635,635.029,81.053,2739.933,1210.74093,986.26807
